In [1]:
from scipy.special import expit #to calculate logit 
import scipy.io as si
import numpy as np

#.mat file has two matrix X (5000*400) and y(5000*1)
File = si.loadmat('ex4data1.mat')

#sigmoid function implementation
def sigmoid_function(z):
          return expit(z)

#sigmoid gradient implementation
def sigmoid_gradient(Z):
    temp = sigmoid_function(Z)
    return (temp*(1 - temp))

#implement Forward Propagation
def compute_input_layer(theta1,input_layer):
    return (sigmoid_function(np.dot(input_layer,theta1.T)))

def compute_hidden_layer(theta2,A):
    return (sigmoid_function(np.dot(A,theta2.T)))

#return 5000*10 matrix
def compute_hypothesis(Theta1,Theta2,input_layer):
    
    #get 5000*25 matrix
    A1 = compute_input_layer(Theta1,input_layer)
    
    #change A dim to 5000*26
    A1 = np.insert(A1,0,1,axis = 1)

    #get 5000*10 matrix
    A2 = compute_hidden_layer(Theta2,A1)
    return A2 , A1

#implement Cost function for Neural Nertwork with no Regularization

#calculate cost for a given training data and parameters
def Cost_function_n_reg(theta1,theta2,input_layer,actual_result,K):
    
    #calculate hypothesis
    hypothesis1 , hypothesis2 = compute_hypothesis(theta1,theta2,input_layer)
    
    
    term1 =  - np.sum(np.multiply(actual_result,np.log(hypothesis1))) 
    term2 =  - np.sum(np.multiply((1 -actual_result),np.log(1 - hypothesis1))) 
    cost = term1 + term2
            
    cost = (cost/(len(hypothesis1)))
    return cost


In [2]:
print(sigmoid_gradient(np.array([-1, -0.5 ,0, 0.5, 1])))

[ 0.19661193  0.23500371  0.25        0.23500371  0.19661193]


In [3]:
#load input_layer
input_layer = np.array(File['X'])
input_layer = np.insert(input_layer,0,1,axis = 1)

#randomly initializes theata1(25,401) and theta2(10,26) for a given epsilon


epsilon = 0.12
init_theta1 = np.random.uniform(-epsilon,epsilon,(25,401))
init_theta2 = np.random.uniform(-epsilon,epsilon,(10,26))

In [4]:
print(init_theta1.shape)

(25, 401)


In [5]:
#create actual result of training set in form of {0,1}
def result_binary(actual_result,num_labels):
    y = np.zeros((len(actual_result),num_labels))
    for i in range(len(actual_result)):
        temp = actual_result[i]
        y[i][temp - 1] = 1
    return y

output_layer = result_binary(File['y'],10)
print(output_layer[4999])

[ 0.  0.  0.  0.  0.  0.  0.  0.  1.  0.]


In [6]:
hypothesis1 , hypothesis2 = compute_hypothesis(init_theta1,init_theta2,input_layer)
delta_3 = hypothesis1 - output_layer
print(delta_3.shape)

(5000, 10)


In [7]:
term1 = np.dot(delta_3,init_theta2).T
term1 = np.delete(term1,0,0)
delta_2 = np.multiply(term1.T,sigmoid_gradient(np.dot(input_layer,init_theta1.T)))

print(delta_2.shape)

(5000, 25)


In [8]:
grad_1 = np.dot(input_layer.T,delta_2).T
grad_1.shape

(25, 401)

In [9]:
print(hypothesis2.shape)
print(delta_3.shape)
grad_2 = np.dot(hypothesis2.T,delta_3)
grad_2 = np.delete(grad_2,0,0).T
print(grad_2.shape)

(5000, 26)
(5000, 10)
(10, 25)


In [11]:
#implement BackPropagation
def BackPropagation(init_theta1,init_theta2,input_layer,output_layer,m):
    
    #calculate forward propagation
    hypothesis1 , hypothesis2 = compute_hypothesis(init_theta1,init_theta2,input_layer)
    
    #computes small delta
    
    #compute delta_3
    delta_3 = hypothesis1 - output_layer
    
    #compute delta_2
    term1 = delta_3.dot(init_theta2)
    delta_2 = term1*(hypothesis2*(1 - hypothesis2))
    delta_2 = np.delete(delta_2,0,1)
    
    #compute gradient for layer 1
    grad_1 = np.dot(input_layer.T,delta_2).T
    grad_1 = grad_1/m
    
    #compute gradient for layer 2
    grad_2 = np.dot(hypothesis2.T,delta_3).T
    grad_2 = grad_2/m
    
    return (grad_1 , grad_2)

GRADIENT_1 , GRADIENT_2 = BackPropagation(init_theta1,init_theta2,input_layer,output_layer,len(File['X']))

In [12]:
print(GRADIENT_1.shape,GRADIENT_2.shape)

(25, 401) (10, 26)


In [13]:
#perform  Gradient Checking on a small neural network

#make a small neural network

#define sizes
input_layer_size = 3 #including bias unit
hidden_layer_size = 5 #including bias unit
num_labels = 3
m = 5
def debugInitializeWeights(fan_out,fan_in):
    W = np.zeros((fan_out,fan_in + 1))
    W = (np.sin(np.arange(1,np.size(W) + 1)).reshape(W.shape))/10
    return W
#put some random values
#randomly  initializes test_theta's with a range of (-.12,.12)
epsilon = 0.12
test_theta1 = debugInitializeWeights(hidden_layer_size ,input_layer_size )
test_theta2 = debugInitializeWeights(num_labels,hidden_layer_size )

#randomly generate input layer (5*2)
test_input_layer = debugInitializeWeights(m,input_layer_size - 1)
test_input_layer = np.insert(test_input_layer,0,1,axis = 1)
print(test_input_layer)

#randomly generate  actual result 
test_result = 1 + np.remainder(np.arange(1,m+1),num_labels)
print(test_result)

#create test actualresult in binary form
test_actual_result = np.zeros((m,num_labels))
for i in range(m):
         test_actual_result[i,test_result[i] - 1] = 1

print(test_actual_result)

#make a long test_THETA VECTOR 

print(test_theta1.shape)
print(test_theta2.shape)

[[ 1.          0.0841471   0.09092974  0.014112  ]
 [ 1.         -0.07568025 -0.09589243 -0.02794155]
 [ 1.          0.06569866  0.09893582  0.04121185]
 [ 1.         -0.05440211 -0.09999902 -0.05365729]
 [ 1.          0.0420167   0.09906074  0.06502878]]
[2 3 1 2 3]
[[ 0.  1.  0.]
 [ 0.  0.  1.]
 [ 1.  0.  0.]
 [ 0.  1.  0.]
 [ 0.  0.  1.]]
(5, 4)
(3, 6)


In [14]:
#make calls to backpropagation and Cost_function for small neural network
TEST_GRADIENT_1 , TEST_GRADIENT_2 = BackPropagation(test_theta1,test_theta2,test_input_layer,test_actual_result,m)

In [21]:
def gradient_checking(test_theta1,test_theta2,epsilon,num_labels,test_input_layer,test_result):
    
    GRADIENT_CHECK = np.zeros((len(test_theta1.flatten()) + len(test_theta2.flatten()),1))
    test_theta_L = np.concatenate((test_theta1.flatten(),test_theta2.flatten()),0)
    test_theta_L = test_theta_L.reshape(len(test_theta1.flatten()) + len(test_theta2.flatten()),1)
    
    for i in range(len(test_theta_L)):
        inc = np.zeros((len(test_theta_L),1))
        inc[i] = inc[i] + epsilon
    
        theta_plus = test_theta_L + inc
        theta_minus = test_theta_L - inc
    
        #unrolling for theta_plus
        pass_theta1 = np.array(theta_plus[0:test_theta1.shape[0]*test_theta1.shape[1]]).reshape(test_theta1.shape)
        pass_theta2 = np.array(theta_plus[test_theta1.shape[0]*test_theta1.shape[1]:]).reshape(test_theta2.shape)
    
        cost_plus = Cost_function_n_reg(pass_theta1,pass_theta2,test_input_layer,test_result,num_labels)
    
        #unrolling for theta_minus
        pass_theta1 = np.array(theta_minus[0:test_theta1.shape[0]*test_theta1.shape[1]]).reshape(test_theta1.shape)
        pass_theta2 = np.array(theta_minus[test_theta1.shape[0]*test_theta1.shape[1]:]).reshape(test_theta2.shape)
    
        cost_minus = Cost_function_n_reg(pass_theta1,pass_theta2,test_input_layer,test_result,num_labels)
    
        GRADIENT_CHECK[i] = (cost_plus - cost_minus)/(2*epsilon)
        
    return GRADIENT_CHECK


In [22]:
#make calls to backpropagation and Cost_function for small neural network
TEST_GRADIENT_1 , TEST_GRADIENT_2 = BackPropagation(test_theta1,test_theta2,test_input_layer,test_actual_result,m)

GRADIENT_CHECK = gradient_checking(test_theta1,test_theta2,.0001,num_labels,test_input_layer,test_actual_result)
#COMPARE OUTPUT OF GRADIENT_CHECKING AND BACKPROPAGATION

GRADIENT_CHECK1 = np.array(GRADIENT_CHECK[0:test_theta1.shape[0]*test_theta1.shape[1]]).reshape(test_theta1.shape)
GRADIENT_CHECK2 = np.array(GRADIENT_CHECK[test_theta1.shape[0]*test_theta1.shape[1]:]).reshape(test_theta2.shape)

print("output from Backpropagtion for theta 1 \n")
print(TEST_GRADIENT_1)
print("\noutput from Gradient Checking for theta 1 \n")
print(GRADIENT_CHECK1)

print("\noutput from Backpropagtion for theta 2 \n")
print(TEST_GRADIENT_2)
print("\noutput from Gradient Checking for theta 2 \n")
print(GRADIENT_CHECK2)

output from Backpropagtion for theta 1 

[[  1.23162247e-02   1.73828184e-04   2.61455144e-04   1.08701450e-04]
 [  3.92471369e-03   1.90101252e-04   2.22272331e-04   5.00872547e-05]
 [ -8.08459407e-03   3.13170587e-05  -2.17840341e-05  -5.48569864e-05]
 [ -1.26669105e-02  -1.56130210e-04  -2.45506163e-04  -1.09164881e-04]
 [ -5.59342547e-03  -2.00036572e-04  -2.43630220e-04  -6.32313673e-05]]

output from Gradient Checking for theta 1 

[[  1.23162247e-02   1.73828185e-04   2.61455142e-04   1.08701450e-04]
 [  3.92471368e-03   1.90101255e-04   2.22272329e-04   5.00872543e-05]
 [ -8.08459406e-03   3.13170601e-05  -2.17840368e-05  -5.48569856e-05]
 [ -1.26669105e-02  -1.56130211e-04  -2.45506164e-04  -1.09164882e-04]
 [ -5.59342546e-03  -2.00036570e-04  -2.43630223e-04  -6.32313690e-05]]

output from Backpropagtion for theta 2 

[[ 0.30934772  0.16106714  0.14703652  0.15826858  0.15761671  0.14723636]
 [ 0.108133    0.05616337  0.05195105  0.05473534  0.05530828  0.05177526]
 [ 0.10627

In [ ]:
#learn parameters 